## Finde five five-letter words with 25 different characters

### First approach

No optimization. Every word has the same value, despite how odd it is.

#### Loading the needed modules

In [8]:
import pulp as plp
import pandas as pd
import numpy as np

#### Loading and processing the word list

Source of word list:  

In [2]:
# loading words
file = 'words_alpha.txt'
df = pd.read_csv(file)
df.columns = ['word']

# get length of each word
df['length'] = df['word'].apply(lambda x: len(str(x)))

# keep only words of length 5
dff = df[df.length == 5].copy()

# get number of unique characters of each word
dff['length'] = dff['word'].apply(lambda x: len(set([char for char in x])))

# optional: filter out words which do not contain vows
dff = dff[dff['word'].str.contains('a|e|i|o|u|y')]

# keep only words with 5 unique characters
word_list = dff[dff.length == 5]['word'].values

# return length of lists
print(f'Original list has length {len(df)}')
print(f'Filtered list of five letter words has length {len(dff)}')
print(f'Filtered list with unique five letter characters has length {len(word_list)}')

Original list has length 370104
Filtered list of five letter words has length 15912
Filtered list with unique five letter characters has length 10172


#### Solving the ILP problem

This approach gives a possible of potentially many solutions

In [3]:
# set variables
variables = plp.LpVariable.dict('words', (word_list), lowBound=0, upBound=1, cat='Binary')

# initialize problem
prob = plp.LpProblem('Find_five_words')

# constraint 1: only five words
prob += plp.lpSum([variables[word] for word in word_list]) == 5

# constraint 2: each char is allowed to occur only once
for char in 'abcdefghijklmnopqrstuvwxyz':
    prob += plp.lpSum([variables[word] for word in word_list if char in word]) <= 1

prob.solve()

# returning the solution
solution = [word for word in word_list if variables[word].value() == 1.0]

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/585e79d08bec4dc1ac861218661dfb6d-pulp.lp -o /tmp/585e79d08bec4dc1ac861218661dfb6d-pulp.sol
Reading problem data from '/tmp/585e79d08bec4dc1ac861218661dfb6d-pulp.lp'...
27 rows, 10173 columns, 61032 non-zeros
10172 integer variables, all of which are binary
22398 lines were read
GLPK Integer Optimizer 5.0
27 rows, 10173 columns, 61032 non-zeros
10172 integer variables, all of which are binary
Preprocessing...
27 rows, 10172 columns, 61032 non-zeros
10172 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 27
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
27 rows, 10172 columns, 61032 non-zeros
      0: obj =   0.000000000e+00 inf =   2.400e+01 (6)
     57: obj =   0.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Intege

In [4]:
solution

['frack', 'jowly', 'pbxes', 'vingt', 'zhmud']

## Getting more interesting solutions

### Using Pulp for optimization

### Load a word frequency dataset
Loading Google's NGram dataset to retrieve the word frequencies for optimization

In [5]:
# load Google Books Ngram dataset from Kaggle
file_freq = 'ngram_freq.csv'
# file_freq = 'unigram_freq.csv'
df_freq = pd.read_csv(file_freq)

# get length of words
df_freq['length'] = df_freq['word'].apply(lambda x: len(str(x)))
dff_freq = df_freq[df_freq['length'] == 5].copy()

# get number of unique characters of each word
dff_freq['length'] = dff_freq['word'].apply(lambda x: len(set([char for char in x])))

# keep only words with 5 unique characters
dfff_freq = dff_freq[dff_freq.length == 5].reset_index(drop=True)

# transfer data into dictionary for later look up
freq_dict = {dfff_freq.loc[i, 'word']: dfff_freq.loc[i, 'count'] for i in dfff_freq.index}

# return length of lists
print(f'Original list has length {len(df_freq)}')
print(f'Filtered list with unique five letter characters has length {len(freq_dict)}')

Original list has length 9244879
Filtered list with unique five letter characters has length 427383


#### Adding the frequencies to the word list

Now we match the words of the original list with the count frequencies found in the NGram dataset. Word which are not included in the NGram dataset are given the count 1.

In [16]:
word_dict = {}
not_found_count = 0
for word in word_list:
    try: 
        # try to get counts from NGram dataset
        count = freq_dict[word]
        word_dict[word] = count
    except KeyError:
        # if key is not found in NGram dataset, add it to list with a count of 1
        word_dict[word] = 1
        not_found_count += 1

word_list = list(word_dict.keys())
len(word_dict), not_found_count

(10172, 119)

#### Solve the adapted problem

Finding the optimal solution to the problem. We take the logarithm of the count frequencies to get better solutions since the frequencies range over several magnitutes. After finding an optimal solution which maximizes the word frequency, we exclude the word with the highes frequency count from the problem and re-run the code.

In [13]:
# set variables
variables = plp.LpVariable.dict('words', (word_list), lowBound=0, upBound=1, cat='Binary')

# initialize problem
prob = plp.LpProblem('Find_five_words', plp.LpMaximize)

# objective function
prob += plp.lpSum([variables[word] * np.log(word_dict[word]) for word in word_list])

# constraint 1: only five words
prob += plp.lpSum([variables[word] for word in word_list]) == 5

# constraint 2: each char is allowed to occur only once
for char in 'abcdefghijklmnopqrstuvwxyz':
    prob += plp.lpSum([variables[word] for word in word_list if char in word]) <= 1
          
solutions = []
   
# continue solving while not all solutions are found
while True:
    prob.solve()
    
    # break if ILP solution is not optimal
    if plp.LpStatus[prob.status] != "Optimal":
        break
        
    # retrieve results       
    results = [word for word in word_list if variables[word].value() == 1.0]
     
    # find result with most counts for exclusion
    results_dict = {r: word_dict[r] for r in results}        
    exclude = max(results_dict, key=results_dict.get)
        
    # exclude word from problem
    prob += plp.lpSum(variables[exclude]) == 0 
    
    # append new results to solutions list
    solutions.append(results)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/9bc641a9ffb04d1d85a9fd3db13c7aca-pulp.lp -o /tmp/9bc641a9ffb04d1d85a9fd3db13c7aca-pulp.sol
Reading problem data from '/tmp/9bc641a9ffb04d1d85a9fd3db13c7aca-pulp.lp'...
27 rows, 10172 columns, 61032 non-zeros
10172 integer variables, all of which are binary
27422 lines were read
GLPK Integer Optimizer 5.0
27 rows, 10172 columns, 61032 non-zeros
10172 integer variables, all of which are binary
Preprocessing...
27 rows, 10172 columns, 61032 non-zeros
10172 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 27
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
27 rows, 10172 columns, 61032 non-zeros
      0: obj =  -0.000000000e+00 inf =   2.400e+01 (6)
     46: obj =   5.092732070e+01 inf =   6.277e-18 (0)
*    84: obj =   7.097035546e+01

32 rows, 10172 columns, 61037 non-zeros
10172 integer variables, all of which are binary
27427 lines were read
GLPK Integer Optimizer 5.0
32 rows, 10172 columns, 61037 non-zeros
10172 integer variables, all of which are binary
Preprocessing...
27 rows, 10167 columns, 61002 non-zeros
10167 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 27
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
27 rows, 10167 columns, 61002 non-zeros
      0: obj =  -0.000000000e+00 inf =   2.400e+01 (6)
     42: obj =   5.147437851e+01 inf =   0.000e+00 (0)
*    77: obj =   7.006814190e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+    77: mip =     not found yet <=              +inf        (1; 0)
+   625: >>>>>   4.866489231e+01 <=   6.623939694e+01  36.1% (26; 12)
+  

In [12]:
solutions

[['dwarf', 'glyph', 'jocks', 'muntz', 'vibex'],
 ['breck', 'flows', 'japyx', 'vingt', 'zhmud'],
 ['fjord', 'glyph', 'muntz', 'vibex', 'wacks'],
 ['brews', 'flock', 'japyx', 'vingt', 'zhmud'],
 ['brows', 'fleck', 'japyx', 'vingt', 'zhmud'],
 ['breck', 'fowls', 'japyx', 'vingt', 'zhmud'],
 ['blows', 'freck', 'japyx', 'vingt', 'zhmud'],
 ['bowls', 'freck', 'japyx', 'vingt', 'zhmud'],
 ['breck', 'japyx', 'vingt', 'wolfs', 'zhmud'],
 ['dhikr', 'expwy', 'fultz', 'gconv', 'jambs'],
 ['brock', 'flews', 'japyx', 'vingt', 'zhmud'],
 ['flong', 'japyx', 'twick', 'verbs', 'zhmud'],
 ['dumbs', 'fritz', 'gconv', 'japyx', 'whelk'],
 ['becks', 'fultz', 'japyx', 'mordv', 'whing'],
 ['frack', 'jowly', 'pbxes', 'vingt', 'zhmud'],
 ['blitz', 'fconv', 'gryph', 'judex', 'mawks'],
 ['block', 'fremt', 'japyx', 'vughs', 'windz'],
 ['bovld', 'freck', 'japyx', 'muntz', 'whigs'],
 ['chivw', 'expdt', 'flank', 'grosz', 'jumby'],
 ['flong', 'jarvy', 'pbxes', 'twick', 'zhmud'],
 ['expdt', 'gconv', 'jumby', 'whilk', 'z